# Simulation of naive betting strategies

This notebook will demonstrate how to create simple simulations of Premier League seasons using different betting strategies.

In [1]:
import pandas as pd
import numpy as np
import datetime 

### Import and clean data
We will use 9 PL seasons in this example

In [2]:
training_seasons = ["pl1819","pl1718","pl1617","pl1516","pl1415","pl1314","pl1213","pl1112","pl1011"]
training_seasons.reverse()

In [3]:
df = pd.read_csv("data/odds/pl/" + training_seasons[0] + ".csv")
for season in training_seasons[1:]:
    df = pd.concat([df,pd.read_csv("data/odds/pl/" + season + ".csv")], sort=False)

In [4]:
display(df.head())
df.shape

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,BbMxAHH,BbAvAHH,BbMxAHA,BbAvAHA,PSH,PSD,PSA,PSCH,PSCD,PSCA
0,E0,14/08/10,Aston Villa,West Ham,3.0,0.0,H,2.0,0.0,H,...,1.45,1.40,3.00,2.78,NaN,NaN,NaN,NaN,NaN,NaN
1,E0,14/08/10,Blackburn,Everton,1.0,0.0,H,1.0,0.0,H,...,2.20,2.05,1.85,1.75,NaN,NaN,NaN,NaN,NaN,NaN
2,E0,14/08/10,Bolton,Fulham,0.0,0.0,D,0.0,0.0,D,...,1.60,1.55,2.56,2.36,NaN,NaN,NaN,NaN,NaN,NaN
3,E0,14/08/10,Chelsea,West Brom,6.0,0.0,H,2.0,0.0,H,...,2.06,2.01,1.91,1.85,NaN,NaN,NaN,NaN,NaN,NaN
4,E0,14/08/10,Sunderland,Birmingham,2.0,2.0,D,1.0,0.0,H,...,1.64,1.55,2.50,2.38,NaN,NaN,NaN,NaN,NaN,NaN


(3421, 77)

In [5]:
def filter_team(df,name):
    return df[(df.HomeTeam == name) | (df.AwayTeam == name)].sort_index()

def home_games(df,name):
    return df[df.HomeTeam == name]

def away_games(df,name):
    return df[df.AwayTeam == name]

#Season format = xx/xx
def filter_season(df,season):
    return df.loc['20'+season.split("/")[0] + '-07-15':'20' + season.split("/")[1] + '-07-12'].sort_index()

def lastx_games(df,x):
     return filter_team(df[:5],row["HomeTeam"])[-x:] 
    
def get_teams(df):
    #array of teams this season
    return df.HomeTeam.unique()

Set the match dates as index. This needs to be done in order to simulate seasons in correct date order

In [6]:
df['Date'] = pd.to_datetime(df['Date'])
df.index = df.Date

In [7]:
filter_season(df,"15/16").head()

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,BbMxAHH,BbAvAHH,BbMxAHA,BbAvAHA,PSH,PSD,PSA,PSCH,PSCD,PSCA
Date,,,,,,,,,,,,,,,,,,,,,
2015-08-02,E0,2015-08-02,West Ham,Man United,1.0,1.0,D,0.0,0.0,D,...,1.97,1.91,2.00,1.95,4.10,3.72,1.97,4.30,3.71,1.93
2015-08-02,E0,2015-08-02,Newcastle,Stoke,1.0,1.0,D,0.0,0.0,D,...,1.99,1.94,1.98,1.93,2.28,3.34,3.54,2.26,3.38,3.54
2015-08-02,E0,2015-08-02,Burnley,West Brom,2.0,2.0,D,2.0,1.0,H,...,1.83,1.78,2.15,2.08,2.60,3.28,3.02,2.53,3.14,3.25
2015-08-08,E0,2015-08-08,Man United,Tottenham,1.0,0.0,H,1.0,0.0,H,...,2.20,2.09,1.82,1.78,1.65,4.09,5.90,1.64,4.07,6.04
2015-08-08,E0,2015-08-08,Norwich,Crystal Palace,1.0,3.0,A,0.0,1.0,A,...,1.83,1.78,2.17,2.08,2.52,3.35,3.08,2.46,3.39,3.14


Define some useful columns

In [8]:
match_odds_cols = ["B365H","B365D","B365A"]
goals_odds_cols = ["BbAv>2.5", "BbAv<2.5"]
"""
FTHG and HG = Full Time Home Team Goals
FTAG and AG = Full Time Away Team Goals
HS = Home Team Shots
AS = Away Team Shots
HST = Home Team Shots on Target
AST = Away Team Shots on Target
"""
match_stat_cols = ["HST","AST","HS","AS","FTHG","FTAG"]
seasons=["10/11","11/12","12/13","13/14","14/15","15/16","16/17","17/18","18/19"]

In [9]:
df[match_odds_cols].idxmin(axis=1).head() #favourites in each game

Date
2010-08-14    B365H
2010-08-14    B365A
2010-08-14    B365H
2010-08-14    B365H
2010-08-14    B365H
dtype: object

### Simulate the seasons
simulate_seasons will iterate over all the seasons and place bets based on the favourite (min) or underdrog (max). The simulation will only bet on one match at a time, and refill the balance by 100 each time it looses. This is just a dummy example. A more realistic approach would be to group bets during the same day.

In [10]:
outcomes = ["H","D","A"]


def simulate_seasons(strategy):
    for season in seasons:
        seasonal_statistics = {}
        seasonal_bets = {}
        df_season = filter_season(df,season)
        teams = get_teams(df_season)

        balance = 100
        money_spent = 100
        max_balance = 0

        for team in teams:
            seasonal_statistics[team] = {
                "s": 0, #shots
                "ts": 0, #target shots
                "w": 0, #wins
                "d": 0, #draws
                "l": 0 #loss
            }

            seasonal_bets = {
                "correct": 0,
                "wrong": 0
            }

        print("Season ",season, " with ",strategy, " strategy")
        
        for _,match in df_season.iterrows():
            #.idxmax(axis=1)
            if strategy == "max":
                guess = outcomes[np.argmax(match[match_odds_cols].values)]
                value = np.max(match[match_odds_cols].values)
            if strategy == "min":
                guess = outcomes[np.argmin(match[match_odds_cols].values)]
                value = np.min(match[match_odds_cols].values)
                
            if guess == match.FTR: #Odds went in
                seasonal_bets["correct"] = seasonal_bets["correct"] + 1
                balance = balance * value
                if balance > max_balance:
                    max_balance = balance
            else:
                seasonal_bets["wrong"] = seasonal_bets["wrong"] + 1
                balance = 100 #Set inn 100 when you loose
                money_spent += 100 

        seasonal_bets["money_spent"] = money_spent
        seasonal_bets["max_balance"] = int(max_balance)

        print(seasonal_bets)
        
    
simulate_seasons("min")
print("\n\n")
simulate_seasons("max")

Season  10/11  with  min  strategy
{'correct': 186, 'wrong': 169, 'money_spent': 17000, 'max_balance': 23480}
Season  11/12  with  min  strategy
{'correct': 185, 'wrong': 196, 'money_spent': 19700, 'max_balance': 3079}
Season  12/13  with  min  strategy
{'correct': 209, 'wrong': 175, 'money_spent': 17600, 'max_balance': 3040}
Season  13/14  with  min  strategy
{'correct': 218, 'wrong': 143, 'money_spent': 14400, 'max_balance': 21066}
Season  14/15  with  min  strategy
{'correct': 201, 'wrong': 176, 'money_spent': 17700, 'max_balance': 12745}
Season  15/16  with  min  strategy
{'correct': 190, 'wrong': 208, 'money_spent': 20900, 'max_balance': 3980}
Season  16/17  with  min  strategy
{'correct': 223, 'wrong': 146, 'money_spent': 14700, 'max_balance': 149826}
Season  17/18  with  min  strategy
{'correct': 232, 'wrong': 175, 'money_spent': 17600, 'max_balance': 13139}
Season  18/19  with  min  strategy
{'correct': 190, 'wrong': 129, 'money_spent': 13000, 'max_balance': 51209}



Season  1